Original Notebook:

https://www.kaggle.com/code/taylorsamarel/drw-blend-horizontal-blend-vertical-remix

In [1]:
import pandas as pd
def iBlend(path_to_ds, file_short_names, sls):

    def tida(sls):
        
        def read_subm(sls,i):
            tnm = sls["subm"][i]["name"]
            # Check if this is the submission(47) file
            if tnm == "submission(47)":
                FiN = "/kaggle/input/convergence-drw/" + tnm + ".csv"
            else:
                FiN = sls["path"] + tnm + ".csv"
            return pd.read_csv(FiN).rename(columns={'target':tnm, sls["target"]:tnm})
        
        dfs_subm = [read_subm(sls,i) for i in range(len(sls["subm"]))]
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=['ID'])
        
        for i in range(2, len(sls["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=['ID'])
            
        cols = [col for col in df_subms.columns if col != "ID"]
        short_name_cols = [c.replace(sls["prefix"], '') for c in cols]
        corrects  = [wt for wt in sls["subwts"]]
        weights   = [subm['weight'] for subm in sls["subm"]]
        corrects2 = [wt for wt in sls["subwts2"]]
        weights2  = [subm['weight'] for subm in sls["subm2"]]
        
        def alls(x, cs=cols):
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [
              t[0].replace(sls["prefix"], '')
              for t in sorted(tes,key=lambda k:k[1],reverse=True if sls["sort"]=='desc' else False)]
            return subms_sorted
        
        def correct(x, cs=cols, w=weights, cw=corrects, w2=weights2, cw2=corrects2):
            ic = [x['alls'].index(c) for c in short_name_cols]
            if x['abs(mx-m)'] > 0.74:
                cS = [x[cols[j]] * (w [j] + cw [ic[j]]) for j in range(len(cols))]
            else:
                cS = [x[cols[j]] * (w2[j] + cw2[ic[j]]) for j in range(len(cols))]
            return sum(cS)

        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        df_subms['abs(mx-m)']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        
        df_subms['alls']        = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[sls["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={sls["target"]:"ensemble"})
        
        df_subms.insert(loc=1, column=' _ ', value=['   '] * sls["q_rows"])
        
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.3f}'.format)
        vcols = ['ID'] + [' _ '] + short_name_cols + [' _ '] + ['abs(mx-m)'] + [' _ '] + ['alls'] + [' _ '] + ['ensemble']
        df_subms = df_subms[vcols]
        display(df_subms.head(100))
        pd.set_option('display.float_format', '{:.7f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":sls["target"]})
        
        return df_subms
        

    sample_subm = pd.read_csv(path_to_ds + file_short_names[1] + ".csv")

    
    def ensemble_tida(sls,submission=sample_subm):   
        sls['sort'] = 'desc'
        dfs = tida(sls)
        dfD = dfs[['ID', sls['target']]]
        dfD.to_csv(f'tida_desc.csv', index=False)
        sls['sort'] = 'asc'
        dfs = tida(sls)
        dfA = dfs[['ID', sls['target']]]
        dfA.to_csv(f'tida_asc.csv',  index=False)
        target,d,a = sls['target'],sls['desc'],sls['asc']
        submission[target] = dfD[target] * d + a * dfA[target]
        return submission

    submission = ensemble_tida(sls)
    
    return submission

path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.83975','0.86767','0.89178']

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+1.00, -0.40, -0.60],                          # LB = 0.93598
      'subm'  : [
        { 'name':file_short_names[0],'weight':0.40, },
        { 'name':file_short_names[1],'weight':0.60, },
        { 'name':file_short_names[2],'weight':1.00, },
      ]
    }


path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.88377','0.89178','submission(47)']  # Replaced 0.90038

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.45,
      'asc'   : 0.55,
      'subwts': [+0.55, -0.20, -0.35],                          # LB = 0.89864
      'subm'  : [
        { 'name':file_short_names[0],'weight':0.27, },
        { 'name':file_short_names[1],'weight':0.33, },
        { 'name':file_short_names[2],'weight':0.40, },
      ]
    }


path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.88377','0.89178','submission(47)']  # Replaced 0.90038

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.20,
      'asc'   : 0.80,
      'subwts': [+0.50, -0.20, -0.30],                          # LB = 0.90174
      'subm'  : [
        { 'name':file_short_names[0],'weight':0.20, },
        { 'name':file_short_names[1],'weight':0.30, },
        { 'name':file_short_names[2],'weight':0.50, },
      ]
    }


path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.83975','0.86767','0.88377','0.89178','submission(47)']  # Replaced 0.90038

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+0.40, +0.05, -0.05,-0.15,-0.25],              # LB = 0.93794
      'subm'  : [
         { 'name':file_short_names[0],'weight':0.20, },
         { 'name':file_short_names[1],'weight':0.30, },
         { 'name':file_short_names[2],'weight':0.01, },
         { 'name':file_short_names[3],'weight':0.47, },
         { 'name':file_short_names[4],'weight':0.02, },
      ]
    }

path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.83975','0.86767','0.88377','0.89178','submission(47)']  # Replaced 0.90038

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+0.20, +0.10, -0.05,-0.10,-0.15],              # LB = 0.94828
      'subm'  : [
         { 'name':file_short_names[0],'weight':0.20, },
         { 'name':file_short_names[1],'weight':0.20, },
         { 'name':file_short_names[2],'weight':0.20, },
         { 'name':file_short_names[3],'weight':0.20, },
         { 'name':file_short_names[4],'weight':0.21, },
      ]
    }


params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+0.20, +0.10, -0.05,-0.10,-0.15],              # LB = 0.94857
      'subm'  : [
         { 'name':file_short_names[0],'weight':0.20, },
         { 'name':file_short_names[1],'weight':0.20, },
         { 'name':file_short_names[2],'weight':0.21, },
         { 'name':file_short_names[3],'weight':0.22, },
         { 'name':file_short_names[4],'weight':0.235, },
      ],
      'subwts2': [+0.18, +0.09, -0.04,-0.09,-0.14],             # LB = ?
      'subm2'  : [
         { 'name':file_short_names[0],'weight':0.19, },
         { 'name':file_short_names[1],'weight':0.20, },
         { 'name':file_short_names[2],'weight':0.21, },
         { 'name':file_short_names[3],'weight':0.22, },
         { 'name':file_short_names[4],'weight':0.235, },
      ]
    }

df = iBlend ( path_to_ds, file_short_names, params )

df.to_csv('submission.csv', index=False)

display(df)

,ID,_,0.83975,0.86767,0.88377,0.89178,submission(47),_,abs(mx-m),_,alls,_,ensemble
0,1,,0.009,-0.107,-0.190,-0.191,-0.188,,0.200,,"[0.83975, 0.86767, submission(47), 0.88377, 0....",,-0.102
1,2,,0.249,0.416,0.215,0.196,0.258,,0.219,,"[0.86767, submission(47), 0.83975, 0.88377, 0....",,0.321
2,3,,-1.531,-1.981,-1.099,-0.971,-1.004,,1.010,,"[0.89178, submission(47), 0.88377, 0.83975, 0....",,-1.172
3,4,,-0.236,-0.237,-0.154,-0.157,-0.206,,0.083,,"[0.88377, 0.89178, submission(47), 0.83975, 0....",,-0.187
4,5,,-0.199,0.289,0.173,0.178,0.132,,0.488,,"[0.86767, 0.89178, 0.88377, submission(47), 0....",,0.204
5,6,,-0.169,-0.472,-0.799,-0.766,-0.822,,0.653,,"[0.83975, 0.86767, 0.89178, 0.88377, submissio...",,-0.511
6,7,,-0.388,1.238,1.384,1.976,1.270,,2.364,,"[0.89178, 0.88377, submission(47), 0.86767, 0....",,1.598
7,8,,0.057,0.126,-0.270,-0.326,-0.168,,0.452,,"[0.86767, 0.83975, submission(47), 0.88377, 0....",,-0.027
8,9,,0.247,0.645,0.088,0.077,0.163,,0.568,,"[0.86767, 0.83975, submission(47), 0.88377, 0....",,0.363
9,10,,0.230,0.071,0.212,0.189,0.184,,0.159,,"[0.83975, 0.88377, 0.89178, submission(47), 0....",,0.214


,ID,_,0.83975,0.86767,0.88377,0.89178,submission(47),_,abs(mx-m),_,alls,_,ensemble
0,1,,0.009,-0.107,-0.190,-0.191,-0.188,,0.200,,"[0.89178, 0.88377, submission(47), 0.86767, 0....",,-0.181
1,2,,0.249,0.416,0.215,0.196,0.258,,0.219,,"[0.89178, 0.88377, 0.83975, submission(47), 0....",,0.243
2,3,,-1.531,-1.981,-1.099,-0.971,-1.004,,1.010,,"[0.86767, 0.83975, 0.88377, submission(47), 0....",,-1.631
3,4,,-0.236,-0.237,-0.154,-0.157,-0.206,,0.083,,"[0.86767, 0.83975, submission(47), 0.89178, 0....",,-0.228
4,5,,-0.199,0.289,0.173,0.178,0.132,,0.488,,"[0.83975, submission(47), 0.88377, 0.89178, 0....",,0.039
5,6,,-0.169,-0.472,-0.799,-0.766,-0.822,,0.653,,"[submission(47), 0.88377, 0.89178, 0.86767, 0....",,-0.779
6,7,,-0.388,1.238,1.384,1.976,1.270,,2.364,,"[0.83975, 0.86767, submission(47), 0.88377, 0....",,0.742
7,8,,0.057,0.126,-0.270,-0.326,-0.168,,0.452,,"[0.89178, 0.88377, submission(47), 0.83975, 0....",,-0.231
8,9,,0.247,0.645,0.088,0.077,0.163,,0.568,,"[0.89178, 0.88377, submission(47), 0.83975, 0....",,0.152
9,10,,0.230,0.071,0.212,0.189,0.184,,0.159,,"[0.86767, submission(47), 0.89178, 0.88377, 0....",,0.158


,ID,prediction
0,1,-0.1576531
1,2,0.2663011
2,3,-1.4933765
3,4,-0.2153350
4,5,0.0885497
...,...,...
538145,538146,-0.3172647
538146,538147,0.1401370
538147,538148,-0.9600113
538148,538149,0.8098949


Result:

Score: 0.94980

Rank: 17 (2025-07-17-12:51, JST)

Runtime: 7min

Your Best Entry!
Your most recent submission scored 0.94980, which is an improvement of your previous score of 0.94857. Great job!

Rank 17. A small price to pay for my social life. #kaggle - https://kaggle.com/competitions/drw-crypto-market-prediction 

DRW Trading Group. DRW - Crypto Market Prediction. https://kaggle.com/competitions/drw-crypto-market-prediction, 2025. Kaggle.